<a href="https://colab.research.google.com/github/zeitgeist-hash/GV918-Week04/blob/main/Week_04_Class_Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description

In this exercise, we will continue the analysis of UK Political Data, which I presented in the lecture. We have already conducted merging Election and Brexit referendum data. We will work with additional datasets, coming from: 

https://commonslibrary.parliament.uk/constituency-dashboard/

In particular, we will use the following data:

- Population
- Unemployment
- Housing price

# Clone a data repository

As we will use the data for several weeks, I decided to create a separate repository for the data. We can clone the reposiotory to Colab working directory to start working on the data.

In [51]:
!git clone  https://github.com/University-of-Essex-Dept-of-Government/GV918-UK-politics-data

fatal: destination path 'GV918-UK-politics-data' already exists and is not an empty directory.


In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Rerun the code in the lecture

In [53]:
import pandas as pd
import numpy as np


In [54]:

df_elec = pd.read_excel("/content/GV918-UK-politics-data/Data/general-election-results-2019.xlsx", sheet_name = 'voting-summary')

df_brexit = pd.read_csv("/content/GV918-UK-politics-data/Data/brexit-vote.csv")

df_brexit.rename({'PCON11CD':'ons_id'}, axis = 1, inplace=True)

"""## Select columns"""

df_brexit.rename({'Figure to use':'leave_pct'}, axis = 1, inplace=True)

df_brexit_sub = df_brexit[['ons_id', 'leave_pct']]

df_elec['con_pct'] = df_elec['con'] / df_elec['valid_votes']
df_elec['lab_pct'] = df_elec['lab'] / df_elec['valid_votes']

df_elec.head()

"""## Merge"""
# use "left" merge

df_merge = df_elec.merge(df_brexit_sub, how='left', on = "ons_id")


In [55]:
df_merge.shape

(650, 42)

# Unemployment

What we want to get from this file is the unemployment rate (or rate of unemployment claims) in November 2019, right before the election. 

## Read the file

In [56]:
df_unemp = pd.read_excel('/content/GV918-UK-politics-data/Data/Unemployment.xlsx', sheet_name = "Data")

In [57]:
df_unemp.head()

,ONSConstID,ConstituencyName,RegionID,RegionName,CountryID,CountryName,DateThisUpdate,DateOfDataset,UnempConstNumber,UnempConstRate,UnempRegionNumber,UnempRegionRate,UnempCountryNumber,UnempCountryRate,Unnamed: 14
0,E14000554,Berwick-upon-Tweed,E15000001,North East,K02000001,UK,2020-10-13,2010-05-01,1216,0.025744,80105,0.04769,1502155,0.036923,0
1,E14000569,Bishop Auckland,E15000001,North East,K02000001,UK,2020-10-13,2010-05-01,2361,0.042521,80105,0.04769,1502155,0.036923,0
2,E14000574,Blaydon,E15000001,North East,K02000001,UK,2020-10-13,2010-05-01,1909,0.034575,80105,0.04769,1502155,0.036923,0
3,E14000575,Blyth Valley,E15000001,North East,K02000001,UK,2020-10-13,2010-05-01,2248,0.042177,80105,0.04769,1502155,0.036923,0
4,E14000641,City of Durham,E15000001,North East,K02000001,UK,2020-10-13,2010-05-01,1590,0.024718,80105,0.04769,1502155,0.036923,0


In [58]:
df_unemp.value_counts("DateOfDataset").sort_index()

DateOfDataset
2010-05-01    650
2010-06-01    650
2010-07-01    650
2010-08-01    650
2010-09-01    650
             ... 
2020-05-01    650
2020-06-01    650
2020-07-01    650
2020-08-01    650
2020-09-01    650
Length: 135, dtype: int64

## Data wrangling and merge

- For this data, what we need to do is relatively simple
- The steps are:
  - Select the rows for the nearby month (Nov 2019)
  - Keep the columns for ID vars and unemployment rate
  - Merge with the main dataset

In [59]:
df_unemp_sub = df_unemp[df_unemp['DateOfDataset'] == '2019-11-01'][['ONSConstID', 'UnempConstRate']]
df_unemp_sub

,ONSConstID,UnempConstRate
74100,E14000554,0.023341
74101,E14000569,0.042488
74102,E14000574,0.033568
74103,E14000575,0.045496
74104,E14000641,0.021874
...,...,...
74745,N06000014,0.017839
74746,N06000015,0.020793
74747,N06000016,0.023917
74748,N06000017,0.021030


In [60]:
df_merge = df_merge.merge(df_unemp_sub, left_on = 'ons_id', right_on = 'ONSConstID')
df_merge

,ons_id,ons_region_id,constituency_name,county_name,region_name,country_name,constituency_type,declaration_time,mp_firstname,mp_surname,mp_gender,result,first_party,second_party,electorate,valid_votes,invalid_votes,majority,con,lab,ld,brexit,green,snp,pc,dup,sf,sdlp,uup,alliance,other,other_winner,mp_fullname,majority_percent,majority_rank,turnout_const,turnout_reg,turnout_uk,turnout_2017,con_pct,lab_pct,leave_pct,ONSConstID,UnempConstRate
0,W07000049,W92000004,Aberavon,West Glamorgan,Wales,Wales,County,2019-12-13 02:30:00,Stephen,Kinnock,Male,Lab hold,Lab,Con,50750,31598,82,10490,6518,17008,1072,3108,450,0,2711,0,0,0,0,0,731,0,Stephen Kinnock,0.331983,209,0.622621,0.665721,0.673108,0.666800,0.206279,0.538262,0.601245,W07000049,0.036222
1,W07000058,W92000004,Aberconwy,Clwyd,Wales,Wales,County,2019-12-13 03:09:00,Robin,Millar,Male,Con hold,Con,Lab,44699,31865,123,2034,14687,12653,1821,0,0,0,2704,0,0,0,0,0,0,0,Robin Millar,0.063832,569,0.712879,0.665721,0.673108,0.710482,0.460913,0.397081,0.521971,W07000058,0.026173
2,S14000001,S92000003,Aberdeen North,Scotland,Scotland,Scotland,Borough,2019-12-13 03:13:00,Kirsty,Blackman,Female,SNP hold,SNP,Con,62489,37413,72,12670,7535,4939,2846,1008,880,20205,0,0,0,0,0,0,0,0,Kirsty Blackman,0.338652,202,0.598713,0.680736,0.673108,0.591614,0.201401,0.132013,0.430922,S14000001,0.033311
3,S14000002,S92000003,Aberdeen South,Scotland,Scotland,Scotland,Borough,2019-12-13 03:34:00,Stephen,Flynn,Male,SNP gain from Con,SNP,Con,65719,45638,131,3990,16398,3834,5018,0,0,20388,0,0,0,0,0,0,0,0,Stephen Flynn,0.087427,533,0.694441,0.680736,0.673108,0.684887,0.359306,0.084009,0.321431,S14000002,0.019850
4,S14000003,S92000003,Airdrie and Shotts,Scotland,Scotland,Scotland,County,2019-12-13 02:59:00,Neil,Gray,Male,SNP hold,SNP,Lab,64011,39772,91,5201,7011,12728,1419,0,685,17929,0,0,0,0,0,0,0,0,Neil Gray,0.130770,459,0.621331,0.680736,0.673108,0.592430,0.176280,0.320024,0.398381,S14000003,0.040079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,E14001059,E12000002,Wythenshawe and Sale East,Greater Manchester,North West,England,Borough,2019-12-13 05:04:00,Michael,Kane,Male,Lab hold,Lab,Con,76313,44759,125,10396,13459,23855,3111,2717,1559,0,0,0,0,0,0,0,58,0,Michael Kane,0.232266,331,0.586519,0.656236,0.673108,0.600385,0.300699,0.532965,0.496481,E14001059,0.039245
646,E14001060,E12000009,Yeovil,Somerset,South West,England,County,2019-12-13 05:28:00,Marcus,Fysh,Male,Con hold,Con,LD,82468,59260,148,16181,34588,3761,18407,0,1629,0,0,0,0,0,0,0,875,0,Marcus Fysh,0.273051,279,0.718582,0.720191,0.673108,0.716436,0.583665,0.063466,0.598655,E14001060,0.021093
647,W07000041,W92000004,Ynys Mon,Gwynedd,Wales,Wales,County,2019-12-13 02:45:00,Virginia,Crosbie,Female,Con gain from Lab,Con,Lab,51925,36552,121,1968,12959,10991,0,2184,0,0,10418,0,0,0,0,0,0,0,Virginia Crosbie,0.053841,581,0.703938,0.665721,0.673108,0.706090,0.354536,0.300695,0.509420,W07000041,0.029914
648,E14001061,E12000003,York Central,North Yorkshire,Yorkshire and The Humber,England,Borough,2019-12-13 04:18:00,Rachael,Maskell,Female,Lab hold,Lab,Con,74899,49505,198,13545,13767,27312,4149,1479,2107,0,0,0,0,0,0,0,691,0,Rachael Maskell,0.273609,278,0.660957,0.643472,0.673108,0.686646,0.278093,0.551702,0.388207,E14001061,0.015197


# Housing price dataset

We conduct essentially the same. Extract the most recent data in housing price, and merge it with the main dataframe.

## Read the file

In [61]:
df_housing = pd.read_excel("/content/GV918-UK-politics-data/Data/House-prices.xlsx", 
                           sheet_name="Constituency data table")

## Data wrangling and merge

- First let's findout the data frequency. 

In [62]:
df_housing.head()

,ONSConstID,ConstituencyName,RegionID,RegionName,CountryID,CountryName,DateThisUpdate,DateOfDataset,HouseConstMedianPrice,HouseRegionMedianPrice,HouseCountryMedianPrice,ConstPercentChangeOneYr,RegionPercentChangeOneYr,CountryPercentChangeOneYr,ConstPercentChangeFiveYr,RegionPercentChangeFiveYr,CountryPercentChangeFiveYr,ConstWage,ConstRatio,RegWage,RegRatio,CountryWage,CountryRatio,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,E14000554,Berwick-upon-Tweed,E12000001,North East,K04000001,England and Wales,2019-06-28,1995-12-31,48500,42310,54950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E14000569,Bishop Auckland,E12000001,North East,K04000001,England and Wales,2019-06-28,1995-12-31,39500,42310,54950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E14000574,Blaydon,E12000001,North East,K04000001,England and Wales,2019-06-28,1995-12-31,41750,42310,54950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E14000575,Blyth Valley,E12000001,North East,K04000001,England and Wales,2019-06-28,1995-12-31,39250,42310,54950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E14000641,City of Durham,E12000001,North East,K04000001,England and Wales,2019-06-28,1995-12-31,49000,42310,54950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
df_housing.value_counts("DateOfDataset").sort_index()

DateOfDataset
1995-12-31    650
1996-03-31    650
1996-06-30    650
1996-09-30    650
1996-12-31    650
             ... 
2018-12-01    650
2019-03-01    650
2019-06-01    650
2019-09-01    650
2019-12-01    650
Length: 97, dtype: int64

- Select the rows from Sep 2019
- We use the price change at the constituency level

In [64]:
df_housing_sub = df_housing[['ONSConstID', 'ConstPercentChangeOneYr']][df_housing['DateOfDataset'] == '2019-09-01']

In [65]:
df_merge = df_merge.merge(df_housing_sub, left_on="ons_id", right_on = "ONSConstID")

In [66]:
df_merge

,ons_id,ons_region_id,constituency_name,county_name,region_name,country_name,constituency_type,declaration_time,mp_firstname,mp_surname,mp_gender,result,first_party,second_party,electorate,valid_votes,invalid_votes,majority,con,lab,ld,brexit,green,snp,pc,dup,sf,sdlp,uup,alliance,other,other_winner,mp_fullname,majority_percent,majority_rank,turnout_const,turnout_reg,turnout_uk,turnout_2017,con_pct,lab_pct,leave_pct,ONSConstID_x,UnempConstRate,ONSConstID_y,ConstPercentChangeOneYr
0,W07000049,W92000004,Aberavon,West Glamorgan,Wales,Wales,County,2019-12-13 02:30:00,Stephen,Kinnock,Male,Lab hold,Lab,Con,50750,31598,82,10490,6518,17008,1072,3108,450,0,2711,0,0,0,0,0,731,0,Stephen Kinnock,0.331983,209,0.622621,0.665721,0.673108,0.666800,0.206279,0.538262,0.601245,W07000049,0.036222,W07000049,0.0232326
1,W07000058,W92000004,Aberconwy,Clwyd,Wales,Wales,County,2019-12-13 03:09:00,Robin,Millar,Male,Con hold,Con,Lab,44699,31865,123,2034,14687,12653,1821,0,0,0,2704,0,0,0,0,0,0,0,Robin Millar,0.063832,569,0.712879,0.665721,0.673108,0.710482,0.460913,0.397081,0.521971,W07000058,0.026173,W07000058,0.0461538
2,S14000001,S92000003,Aberdeen North,Scotland,Scotland,Scotland,Borough,2019-12-13 03:13:00,Kirsty,Blackman,Female,SNP hold,SNP,Con,62489,37413,72,12670,7535,4939,2846,1008,880,20205,0,0,0,0,0,0,0,0,Kirsty Blackman,0.338652,202,0.598713,0.680736,0.673108,0.591614,0.201401,0.132013,0.430922,S14000001,0.033311,S14000001,-
3,S14000002,S92000003,Aberdeen South,Scotland,Scotland,Scotland,Borough,2019-12-13 03:34:00,Stephen,Flynn,Male,SNP gain from Con,SNP,Con,65719,45638,131,3990,16398,3834,5018,0,0,20388,0,0,0,0,0,0,0,0,Stephen Flynn,0.087427,533,0.694441,0.680736,0.673108,0.684887,0.359306,0.084009,0.321431,S14000002,0.019850,S14000002,-
4,S14000003,S92000003,Airdrie and Shotts,Scotland,Scotland,Scotland,County,2019-12-13 02:59:00,Neil,Gray,Male,SNP hold,SNP,Lab,64011,39772,91,5201,7011,12728,1419,0,685,17929,0,0,0,0,0,0,0,0,Neil Gray,0.130770,459,0.621331,0.680736,0.673108,0.592430,0.176280,0.320024,0.398381,S14000003,0.040079,S14000003,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,E14001059,E12000002,Wythenshawe and Sale East,Greater Manchester,North West,England,Borough,2019-12-13 05:04:00,Michael,Kane,Male,Lab hold,Lab,Con,76313,44759,125,10396,13459,23855,3111,2717,1559,0,0,0,0,0,0,0,58,0,Michael Kane,0.232266,331,0.586519,0.656236,0.673108,0.600385,0.300699,0.532965,0.496481,E14001059,0.039245,E14001059,0.0857143
646,E14001060,E12000009,Yeovil,Somerset,South West,England,County,2019-12-13 05:28:00,Marcus,Fysh,Male,Con hold,Con,LD,82468,59260,148,16181,34588,3761,18407,0,1629,0,0,0,0,0,0,0,875,0,Marcus Fysh,0.273051,279,0.718582,0.720191,0.673108,0.716436,0.583665,0.063466,0.598655,E14001060,0.021093,E14001060,0.0243902
647,W07000041,W92000004,Ynys Mon,Gwynedd,Wales,Wales,County,2019-12-13 02:45:00,Virginia,Crosbie,Female,Con gain from Lab,Con,Lab,51925,36552,121,1968,12959,10991,0,2184,0,0,10418,0,0,0,0,0,0,0,Virginia Crosbie,0.053841,581,0.703938,0.665721,0.673108,0.706090,0.354536,0.300695,0.509420,W07000041,0.029914,W07000041,0.00014708
648,E14001061,E12000003,York Central,North Yorkshire,Yorkshire and The Humber,England,Borough,2019-12-13 04:18:00,Rachael,Maskell,Female,Lab hold,Lab,Con,74899,49505,198,13545,13767,27312,4149,1479,2107,0,0,0,0,0,0,0,691,0,Rachael Maskell,0.273609,278,0.660957,0.643472,0.673108,0.686646,0.278093,0.551702,0.388207,E14001061,0.015197,E14001061,0.0182648


## Population dataset

What we would like to extract from this dataset is the percentage of elderly (over 65). We need to do some data wrangling.

## Read the dataset

In [67]:
df_pop = pd.read_excel("/content/GV918-UK-politics-data/Data/population-by-age.xlsx", 
                       sheet_name = 'Age by year data')

In [68]:
df_pop.head()

,PCON11CD,PCON11NM,RegionID,RegionNM,All Ages,Age_year,Age_pop,Age_percent,Reg_percent,UK_percent
0,E14000538,Aylesbury,E12000008,South East,125629,0,1601,0.012744,0.010438,0.010822
1,E14000538,Aylesbury,E12000008,South East,125629,1,1664,0.013245,0.010968,0.011266
2,E14000538,Aylesbury,E12000008,South East,125629,2,1772,0.014105,0.011386,0.011637
3,E14000538,Aylesbury,E12000008,South East,125629,3,1775,0.014129,0.011892,0.012012
4,E14000538,Aylesbury,E12000008,South East,125629,4,1793,0.014272,0.012033,0.012009


## Data wrangling

- This data has so many rows, but suppose that what we want to know is the population over the age of 65.

- We will clean the data using the following steps
  - Keep rows of year over 65
  - Aggregate the percentage of the population


In [69]:
df_pop_sub = df_pop[df_pop["Age_year"] >= 65][["PCON11CD", "Age_percent"]]

In [70]:
df_pop_sub

,PCON11CD,Age_percent
65,E14000538,0.008867
66,E14000538,0.008756
67,E14000538,0.008525
68,E14000538,0.008461
69,E14000538,0.007960
...,...,...
59145,N06000002,0.003367
59146,N06000002,0.002864
59147,N06000002,0.002580
59148,N06000002,0.002276


In [71]:
df_pop_elderly = df_pop_sub.groupby('PCON11CD').sum().reset_index()
df_pop_elderly

,PCON11CD,Age_percent
0,E14000530,0.154641
1,E14000531,0.233292
2,E14000532,0.188479
3,E14000533,0.217803
4,E14000534,0.280287
...,...,...
645,W07000076,0.191141
646,W07000077,0.202629
647,W07000078,0.207622
648,W07000079,0.157591


## Merge with the main dataset

In [72]:
df_merge = df_merge.merge(df_pop_elderly, left_on='ons_id', right_on = 'PCON11CD')

# Check correlation

- Let's check the correlation between `con_pct`, `lab_pct`, `leave_pct` and merged variables

In [73]:
df_merge.loc[:, ['con_pct', 'lab_pct','leave_pct', 'Age_percent', 'ConstPercentChangeOneYr', 'UnempConstRate']].corr()

,con_pct,lab_pct,leave_pct,Age_percent,UnempConstRate
con_pct,1.000000,-0.411415,0.595458,0.564467,-0.414482
lab_pct,-0.411415,1.000000,-0.150890,-0.593286,0.568125
leave_pct,0.595458,-0.150890,1.000000,0.393101,0.168809
Age_percent,0.564467,-0.593286,0.393101,1.000000,-0.379418
UnempConstRate,-0.414482,0.568125,0.168809,-0.379418,1.000000
